# Exercise 2 : Règles d’associations
## Realise par :   Donovan Daniel Del Valle Jimenez

In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
from apyori import apriori
from collections import Counter

Données : Il consiste à développer l’étude
d’une base de données bancaire à l’aide des techniques d’analyse d’association. Le but de
l’étude est d’identifier des profiles d’épargnants parmi les clients et identifier des non
épargnants qui seraient intéressés d’acheter des produits d’épargne.

In [2]:
def info(data):
    nb_m = data.isnull().sum().sort_values().sort_values()
    ratio_m = (data.isnull().sum()/data.shape[0])
    unique_count = data.nunique()
    duplicate_count = data.count() - unique_count
    manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)
    return pd.DataFrame({ 'Types': data[list(manquant.index.values)].dtypes,
                           'Nb manquants': nb_m,
                          'Ratio manquants%': ratio_m,
                          'Nb unique': unique_count,
                          'Nb dupliques': duplicate_count})

In [3]:
#Chargement des données
data_file = 'bank-data.csv'
data = pd.read_csv(data_file)

In [4]:
#Il y a 600 personnes et 12 attributs
data.shape

(600, 12)

In [5]:
data.head(10)

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
0,ID12101,48,FEMALE,INNER_CITY,17546.00,NO,1,NO,NO,NO,NO,YES
1,ID12102,40,MALE,TOWN,30085.10,YES,3,YES,NO,YES,YES,NO
2,ID12103,51,FEMALE,INNER_CITY,16575.40,YES,0,YES,YES,YES,NO,NO
3,ID12104,23,FEMALE,TOWN,20375.40,YES,3,NO,NO,YES,NO,NO
4,ID12105,57,FEMALE,RURAL,50576.30,YES,0,NO,YES,NO,NO,NO
5,ID12106,57,FEMALE,TOWN,37869.60,YES,2,NO,YES,YES,NO,YES
6,ID12107,22,MALE,RURAL,8877.07,NO,0,NO,NO,YES,NO,YES
7,ID12108,58,MALE,TOWN,24946.60,YES,0,YES,YES,YES,NO,NO
8,ID12109,37,FEMALE,SUBURBAN,25304.30,YES,2,YES,NO,NO,NO,NO
9,ID12110,54,MALE,TOWN,24212.10,YES,2,YES,YES,YES,NO,NO


In [6]:
info(data)
#Tout d'abord, nous vérifions l'intégrité des données. Il n'y a pas de données manquantes.

,Types,Nb manquants,Ratio manquants%,Nb unique,Nb dupliques
id,object,0,0.0,600,0
age,int64,0,0.0,50,550
sex,object,0,0.0,2,598
region,object,0,0.0,4,596
income,float64,0,0.0,599,1
married,object,0,0.0,2,598
children,int64,0,0.0,4,596
car,object,0,0.0,2,598
save_act,object,0,0.0,2,598
current_act,object,0,0.0,2,598


Nous devrons effectuer certaines étapes de prétraitement pour nettoyer les données. Premièrement, puisque le revenu et l'âge des clients sont des variables numériques continues, nous les convertirions dans une variable catégorielle avec des sous-niveaux définis. Ensuite, les colonnes « ID », âge et revenu sont supprimées de l'ensemble de données d'origine, car elles ne sont pas nécessaires pour une analyse ultérieure.

In [7]:
data['age_cat'] = pd.cut(x=data['age'], bins=[1, 20, 40, 60, 80, 100],
                    labels=['1 à 20', '21 à 40', '41 à 60',
                            '61 à 80', '81 à 100'])

In [8]:
print(data['age_cat'].unique())

Counter(data['age_cat'])

['41 à 60', '21 à 40', '61 à 80', '1 à 20']
Categories (4, object): ['1 à 20' < '21 à 40' < '41 à 60' < '61 à 80']


Counter({'41 à 60': 229, '21 à 40': 244, '61 à 80': 90, '1 à 20': 37})

In [9]:
data['income_cat'] = pd.cut(x=data['income'], bins=3, labels=["low", "med", "high"])

In [10]:
print(data['income_cat'].unique())

Counter(data['income_cat'])

['low', 'med', 'high']
Categories (3, object): ['low' < 'med' < 'high']


Counter({'low': 285, 'med': 235, 'high': 80})

In [11]:
Counter(data['children'])

Counter({1: 135, 3: 68, 0: 263, 2: 134})

In [12]:
## suppresion des attributs non utiles  
data.drop(["id"], axis = 1, inplace = True)
data.drop(["age"], axis = 1, inplace = True)
data.drop(["income"], axis = 1, inplace = True)

Pour une meilleure gestion du profil, nous convertissons l'attribut Children en binaire, pour indiquer si la personne a ou non des enfants.

Afin d'effectuer l'algorithme Apriori, il est nécessaire de modifier les valeurs des éléments dans chaque colonne. Avant le changement, de nombreuses colonnes disent OUI ou NON. Lors du traitement avec l'algorithme Apriori, il l'identifiera comme le même élément, malgré son appartenance à des attributs différents. Pour cette raison, nous l'avons remplacé par des expressions plus conformes à chaque attribut.

In [13]:
data['married'] = data['married'].replace(['YES'],'MARRIED')
data['married'] = data['married'].replace(['NO'],'NOT MARRIED')
data['car'] = data['car'].replace(['YES'],'CAR')
data['car'] = data['car'].replace(['NO'],'NOT CAR')
data['save_act'] = data['save_act'].replace(['YES'],'SAVING ACCOUNT')
data['save_act'] = data['save_act'].replace(['NO'],'NOT SAVING ACCOUNT')
data['current_act'] = data['current_act'].replace(['YES'],'CURRENT ACCOUNT')
data['current_act'] = data['current_act'].replace(['NO'],'NOT CURRENT ACCOUNT')
data['mortgage'] = data['mortgage'].replace(['YES'],'MORTGAGE')
data['mortgage'] = data['mortgage'].replace(['NO'],'NOT MORTGAGE')
data['pep'] = data['pep'].replace(['YES'],'PEP')
data['pep'] = data['pep'].replace(['NO'],'NOT PEP')
data['children'] = data['children'].replace([1],'CHILDREN')
data['children'] = data['children'].replace([2],'CHILDREN')
data['children'] = data['children'].replace([3],'CHILDREN')
data['children'] = data['children'].replace([0],'NOT CHILDREN')

In [14]:
data.head(10)

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
0,FEMALE,INNER_CITY,NOT MARRIED,CHILDREN,NOT CAR,NOT SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,PEP,41 à 60,low
1,MALE,TOWN,MARRIED,CHILDREN,CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,MORTGAGE,NOT PEP,21 à 40,med
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
3,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
4,FEMALE,RURAL,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,high
5,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,PEP,41 à 60,med
6,MALE,RURAL,NOT MARRIED,NOT CHILDREN,NOT CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,PEP,21 à 40,low
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
8,FEMALE,SUBURBAN,MARRIED,CHILDREN,CAR,NOT SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,med
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low


Ensuite, nous allons séparer les épargnants des non-épargnants, en tenant compte de la condition donnée: Un épargnant est une personne ayant soit un « mortgage » soit un plan « pep ».

In [15]:
epargnants = data.loc[(data['mortgage']=="MORTGAGE") | (data['pep']=="PEP")]
epargnants.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
0,FEMALE,INNER_CITY,NOT MARRIED,CHILDREN,NOT CAR,NOT SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,PEP,41 à 60,low
1,MALE,TOWN,MARRIED,CHILDREN,CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,MORTGAGE,NOT PEP,21 à 40,med
5,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,PEP,41 à 60,med
6,MALE,RURAL,NOT MARRIED,NOT CHILDREN,NOT CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,PEP,21 à 40,low
11,FEMALE,INNER_CITY,NOT MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,MORTGAGE,NOT PEP,41 à 60,med


In [16]:
non_epargnants = data.drop(epargnants.index)
non_epargnants.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
3,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
4,FEMALE,RURAL,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,high
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
8,FEMALE,SUBURBAN,MARRIED,CHILDREN,CAR,NOT SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,med


In [17]:
epargnants.shape

(391, 11)

In [18]:
Counter(epargnants['children'])

Counter({'CHILDREN': 235, 'NOT CHILDREN': 156})

In [19]:
non_epargnants.shape

(209, 11)

Cela nous laisse avec 391 épargnants et 209 non-épargnants.

Ensuite, nous allons séparer les données en transactions, de manière à ce que l'algorithme Apriori, de la bibliothèque Apryori, puisse les traiter.

In [62]:
records = []
for i in range(0, 391):
    records.append([str(epargnants.values[i,j]) for j in range(0, 10)])

Dans les paramètres de l'algorithme, nous indiquons que nous attendons un minimum de confiance de 0,8 et un minimum de 4 éléments dans la combinaison d'éléments. On a testé autres seuils de confiance et autres longueurs de règles, mais ils génèrent beaucoup de règles pour voir.

In [69]:
association_rules = apriori(records, min_confidence=0.5, min_length=4)
association_results = list(association_rules)

In [70]:
print(len(association_results))

891


In [71]:
association_rules = apriori(records, min_confidence=0.8, min_length=4)
association_results = list(association_rules)

In [72]:
print(len(association_results))

228


Nous obtenons 228 règles. Ensuite, nous allons déboguer les règles. Ce que nous recherchons dans la structure A -> B, ce sont ces règles dont B est HYPOTHÈQUE ou PIP, car ce sont les conditions pour être épargnants, c'est-à-dire ce que nous savons déjà être une conséquence de certains facteurs que nous sommes à la recherche.

In [73]:
count = 0
for i in range(len(association_results)):
    LHS=list(association_results[i][2][0][0])
    RHS=list(association_results[i][2][0][1])
    support=association_results[i][1]
    confidence=association_results[i][2][0][2]
    lift=association_results[i][2][0][3]
    if(('MORTGAGE' in RHS or 'PEP' in RHS) and len(RHS)==1 and len(LHS)>2):
        if(confidence>0.9) :
            count = count + 1
            print("LHS:",LHS,"--","RHS:",RHS)
            print("Support:",support)
            print("Confidence:",confidence)
            print("Lift:",lift)
            print(10*"----")
print("Total de régles: "+str(count))

LHS: ['CURRENT ACCOUNT', 'NOT MORTGAGE', '21 à 40'] -- RHS: ['PEP']
Support: 0.11253196930946291
Confidence: 1.0
Lift: 1.427007299270073
----------------------------------------
LHS: ['CHILDREN', 'NOT MORTGAGE', '41 à 60'] -- RHS: ['PEP']
Support: 0.14066496163682865
Confidence: 1.0
Lift: 1.427007299270073
----------------------------------------
LHS: ['NOT MORTGAGE', '41 à 60', 'FEMALE'] -- RHS: ['PEP']
Support: 0.10230179028132992
Confidence: 1.0
Lift: 1.427007299270073
----------------------------------------
LHS: ['NOT MARRIED', 'NOT MORTGAGE', '41 à 60'] -- RHS: ['PEP']
Support: 0.10230179028132992
Confidence: 1.0
Lift: 1.427007299270073
----------------------------------------
LHS: ['SAVING ACCOUNT', 'NOT MORTGAGE', '41 à 60'] -- RHS: ['PEP']
Support: 0.1329923273657289
Confidence: 1.0
Lift: 1.427007299270073
----------------------------------------
LHS: ['CAR', 'NOT MORTGAGE', 'CHILDREN'] -- RHS: ['PEP']
Support: 0.16879795396419436
Confidence: 1.0
Lift: 1.427007299270073
------

Avec les filtres précédents, il reste 64 règles, dont nous allons en choisir 10.

In [74]:
non_epargnants_r1 = non_epargnants.loc[(non_epargnants['married']=="MARRIED") & (non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['current_act']=="CURRENT ACCOUNT") & (non_epargnants['save_act']=="SAVING ACCOUNT")]
non_epargnants_r1.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
10,FEMALE,TOWN,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,high
19,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low


In [75]:
print("['MARRIED', 'NOT MORTGAGE', 'CURRENT ACCOUNT', 'SAVING ACCOUNT'] --> ['PEP']")
print("Total profiles dans la règle 1 pour proposer un PEP: "+str(len(non_epargnants_r1)))

['MARRIED', 'NOT MORTGAGE', 'CURRENT ACCOUNT', 'SAVING ACCOUNT'] --> ['PEP']
Total profiles dans la règle 1 pour proposer un PEP: 91


In [76]:
non_epargnants_r2 = non_epargnants.loc[(non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['sex']=="MALE") & (non_epargnants['current_act']=="CURRENT ACCOUNT") & (non_epargnants['save_act']=="SAVING ACCOUNT")]
non_epargnants_r2.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
19,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
21,MALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
22,MALE,INNER_CITY,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med


In [77]:
print("['NOT MORTGAGE', 'CURRENT ACCOUNT', 'SAVING ACCOUNT', 'MALE'] --> ['PEP']")
print("Total profiles dans la règle 2 pour proposer un PEP: "+str(len(non_epargnants_r2)))

['NOT MORTGAGE', 'CURRENT ACCOUNT', 'SAVING ACCOUNT', 'MALE'] --> ['PEP']
Total profiles dans la règle 2 pour proposer un PEP: 52


In [78]:
non_epargnants_r3 = non_epargnants.loc[(non_epargnants['region']=="INNER_CITY") & (non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['current_act']=="CURRENT ACCOUNT") & (non_epargnants['save_act']=="SAVING ACCOUNT") ]
non_epargnants_r3.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
22,MALE,INNER_CITY,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
24,MALE,INNER_CITY,NOT MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
35,FEMALE,INNER_CITY,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
62,MALE,INNER_CITY,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med


In [79]:
print("['NOT MORTGAGE', 'INNER_CITY', 'CURRENT ACCOUNT', 'SAVING ACCOUNT'] --> ['PEP']")
print("Total profiles dans la règle 3 pour proposer un PEP: "+str(len(non_epargnants_r3)))

['NOT MORTGAGE', 'INNER_CITY', 'CURRENT ACCOUNT', 'SAVING ACCOUNT'] --> ['PEP']
Total profiles dans la règle 3 pour proposer un PEP: 50


In [80]:
non_epargnants_r4 = non_epargnants.loc[(non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['save_act']=="SAVING ACCOUNT") & (non_epargnants['children']=="CHILDREN") & (non_epargnants['car']=="NOT CAR")]
non_epargnants_r4.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
21,MALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
30,FEMALE,RURAL,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,low
31,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,1 à 20,low
35,FEMALE,INNER_CITY,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
62,MALE,INNER_CITY,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med


In [81]:
print("['SAVING ACCOUNT', 'NOT CAR', 'NOT MORTGAGE', 'CHILDREN'] --> ['PEP']")
print("Total profiles dans la règle 4 pour proposer un PEP: "+str(len(non_epargnants_r4)))

['SAVING ACCOUNT', 'NOT CAR', 'NOT MORTGAGE', 'CHILDREN'] --> ['PEP']
Total profiles dans la règle 4 pour proposer un PEP: 29


In [82]:
non_epargnants_r5 = non_epargnants.loc[(non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['save_act']=="SAVING ACCOUNT") & (non_epargnants['children']=="CHILDREN") & (non_epargnants['sex']=="FEMALE")]
non_epargnants_r5.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
30,FEMALE,RURAL,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,low
31,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,1 à 20,low
33,FEMALE,INNER_CITY,NOT MARRIED,CHILDREN,CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,med
35,FEMALE,INNER_CITY,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
49,FEMALE,RURAL,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low


In [83]:
print("['SAVING ACCOUNT', 'NOT MORTGAGE', 'CHILDREN', 'FEMALE'] --> ['PEP']")
print("Total profiles dans la règle 5 pour proposer un PEP: "+str(len(non_epargnants_r5)))

['SAVING ACCOUNT', 'NOT MORTGAGE', 'CHILDREN', 'FEMALE'] --> ['PEP']
Total profiles dans la règle 5 pour proposer un PEP: 32


In [84]:
non_epargnants_r6 = non_epargnants.loc[(non_epargnants['mortgage']=="NOT MORTGAGE") & (non_epargnants['save_act']=="SAVING ACCOUNT") & (non_epargnants['children']=="CHILDREN") & (non_epargnants['sex']=="MALE")]
non_epargnants_r6.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
21,MALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
24,MALE,INNER_CITY,NOT MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
36,MALE,RURAL,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,1 à 20,low
50,MALE,TOWN,NOT MARRIED,CHILDREN,CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low


In [85]:
print("['SAVING ACCOUNT', 'NOT MORTGAGE', 'MALE', 'CHILDREN'] --> ['PEP']")
print("Total profiles dans la règle 6 pour proposer un PEP: "+str(len(non_epargnants_r6)))

['SAVING ACCOUNT', 'NOT MORTGAGE', 'MALE', 'CHILDREN'] --> ['PEP']
Total profiles dans la règle 6 pour proposer un PEP: 36


In [86]:
non_epargnants_r7 = non_epargnants.loc[(non_epargnants['married']=="MARRIED") & (non_epargnants['save_act']=="SAVING ACCOUNT") & (non_epargnants['pep']=="NOT PEP")]
non_epargnants_r7.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
4,FEMALE,RURAL,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,NOT CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,high
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
10,FEMALE,TOWN,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,high


In [87]:
print("['MARRIED', 'SAVING ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']")
print("Total profiles dans la règle 7 pour proposer un MORTGAGE: "+str(len(non_epargnants_r7)))

['MARRIED', 'SAVING ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']
Total profiles dans la règle 7 pour proposer un MORTGAGE: 120


In [88]:
non_epargnants_r8 = non_epargnants.loc[(non_epargnants['sex']=="MALE") & (non_epargnants['save_act']=="SAVING ACCOUNT") & (non_epargnants['pep']=="NOT PEP")]
non_epargnants_r8.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
9,MALE,TOWN,MARRIED,CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
19,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
21,MALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
22,MALE,INNER_CITY,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med


In [89]:
print("['MALE', 'SAVING ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']")
print("Total profiles dans la règle 8 pour proposer un MORTGAGE: "+str(len(non_epargnants_r8)))

['MALE', 'SAVING ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']
Total profiles dans la règle 8 pour proposer un MORTGAGE: 65


In [90]:
non_epargnants_r9 = non_epargnants.loc[(non_epargnants['car']=="NOT CAR") & (non_epargnants['current_act']=="CURRENT ACCOUNT") & (non_epargnants['pep']=="NOT PEP")]
non_epargnants_r9.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
3,FEMALE,TOWN,MARRIED,CHILDREN,NOT CAR,NOT SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
10,FEMALE,TOWN,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,high
21,MALE,TOWN,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
22,MALE,INNER_CITY,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
30,FEMALE,RURAL,MARRIED,CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,low


In [91]:
print("['NOT CAR', 'CURRENT ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']")
print("Total profiles dans la règle 9 pour proposer un MORTGAGE: "+str(len(non_epargnants_r9)))

['NOT CAR', 'CURRENT ACCOUNT', 'NOT PEP'] --> ['MORTGAGE']
Total profiles dans la règle 9 pour proposer un MORTGAGE: 81


In [92]:
non_epargnants_r10 = non_epargnants.loc[(non_epargnants['children']=="NOT CHILDREN") & (non_epargnants['current_act']=="CURRENT ACCOUNT") & (non_epargnants['pep']=="NOT PEP")]
non_epargnants_r10.head()

,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_cat,income_cat
2,FEMALE,INNER_CITY,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,low
7,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med
10,FEMALE,TOWN,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,61 à 80,high
19,MALE,TOWN,MARRIED,NOT CHILDREN,CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,21 à 40,low
22,MALE,INNER_CITY,MARRIED,NOT CHILDREN,NOT CAR,SAVING ACCOUNT,CURRENT ACCOUNT,NOT MORTGAGE,NOT PEP,41 à 60,med


In [93]:
print("['CURRENT ACCOUNT', 'NOT CHILDREN', 'NOT PEP'] --> ['MORTGAGE']")
print("Total profiles dans la règle 10 pour proposer un MORTGAGE: "+str(len(non_epargnants_r10)))

['CURRENT ACCOUNT', 'NOT CHILDREN', 'NOT PEP'] --> ['MORTGAGE']
Total profiles dans la règle 10 pour proposer un MORTGAGE: 82


In [94]:
total_pep = len(non_epargnants_r1)+len(non_epargnants_r2)+len(non_epargnants_r3)+len(non_epargnants_r4)+len(non_epargnants_r5)+len(non_epargnants_r6)
print("Total profiles PEP: "+str(total_pep))
list_unique_total_pep = pd.concat([non_epargnants_r1, non_epargnants_r2, non_epargnants_r3, non_epargnants_r4, non_epargnants_r5, non_epargnants_r6], axis=0)
print("Total des profiles uniques identifiés en appliquant les 6 règles pour un PEP: "+str(len((list_unique_total_pep.index).unique())))
print("Pourcentage d'épargnants potentiels parmi le total des non-épargnants pour un PEP: "+ str(((len((list_unique_total_pep.index).unique()))/209)*100)+"%")

Total profiles PEP: 290
Total des profiles uniques identifiés en appliquant les 6 règles pour un PEP: 125
Pourcentage d'épargnants potentiels parmi le total des non-épargnants pour un PEP: 59.80861244019139%


In [95]:
total_mortgage = len(non_epargnants_r7)+len(non_epargnants_r8)+len(non_epargnants_r9)+len(non_epargnants_r10)
print("Total profiles MORTGAGE: "+str(total_mortgage))
list_unique_total_mortgage = pd.concat([non_epargnants_r7, non_epargnants_r8, non_epargnants_r9, non_epargnants_r10], axis=0)
print("Total des profiles uniques identifiés en appliquant les 4 règles pour MORTGAGE: "+str(len((list_unique_total_mortgage.index).unique())))
print("Pourcentage d'épargnants potentiels parmi le total des non-épargnants pour un MORTGAGE: "+ str(((len((list_unique_total_mortgage.index).unique()))/209)*100)+"%")

Total profiles MORTGAGE: 348
Total des profiles uniques identifiés en appliquant les 4 règles pour MORTGAGE: 174
Pourcentage d'épargnants potentiels parmi le total des non-épargnants pour un MORTGAGE: 83.25358851674642%


In [96]:
list_unique_total = pd.concat([list_unique_total_mortgage, list_unique_total_pep], axis=0)
print("Total des profiles identifiés avec les 10 règles pour proposer une PEP ou un MORTGAGE: "+ str(total_pep+total_mortgage))
print("Total des profiles uniques identifiés en appliquant les 10 règles pour les deux produits: "+str(len((list_unique_total.index).unique())))
print("Pourcentage d'épargnants potentiels parmi le total des non-épargnants: "+ str((len((list_unique_total.index).unique())/209)*100)+"%")

Total des profiles identifiés avec les 10 règles pour proposer une PEP ou un MORTGAGE: 638
Total des profiles uniques identifiés en appliquant les 10 règles pour les deux produits: 181
Pourcentage d'épargnants potentiels parmi le total des non-épargnants: 86.60287081339713%
